### Demo how to use the LLM with DuckDB

In [2]:
import duckdb

duckdb.__version__

'0.9.0'

In [5]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain import HuggingFaceHub, OpenAI
from langchain import PromptTemplate, LLMChain
from sqlalchemy import Column, Integer, Sequence, String, create_engine

from langchain.llms import GPT4All
from langchain import PromptTemplate, LLMChain

from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 1024  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Make sure the model path is correct for your system!
local_llm = LlamaCpp(
    model_path="/home/wei/Developments/llms/llama.cpp/models/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=4096,
    # n_gqa=8, # add this
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
)

# load_dotenv()
# OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# local_path = ("/home/wei/Developments/llms/llama.cpp/models/sqlcoder.Q4_K_M.gguf")

# initialize the LLM and make chain it with the prompts

# local_llm = GPT4All(
#     model=local_path, 
#     backend="llama", 
# )

uri = 'duckdb:///db/cdn_open_data.db'

connect_args = {
        'read_only': True
    }

CONN = create_engine(uri)

db = SQLDatabase.from_uri(
    uri,
    include_tables=['inventory'], 
	sample_rows_in_table_info=3)

toolkit = SQLDatabaseToolkit(db=db, llm=local_llm)

agent_executor = create_sql_agent(
    llm=local_llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

template = """/
You are a SQL Analyst that is querying a database of Canada Open Data Inventory that about all the Canada Open Data from the Government of Canada website.

You will using the uri to connect the database 'cdn_open_data'.

The database you will interact with is 'inventory' table.

Below is a description of the columns, data types, and information in the columns for the 'inventory' table:

The column name ref_number with the data type VARCHAR contains the following information: Unique identifier for every open data
The column name title_en with the data type VARCHAR contains the following information: English title of the open data
The column name title_fr with the data type VARCHAR contains the following information: French title of the open data
The column name description_en with the data type VARCHAR contains the following information: English description of the open data
The column name description_fr with the data type VARCHAR contains the following information: French description of the open data
The column name publisher_en with the data type VARCHAR contains the following information: Publisher name in English
The column name publisher_fr with the data type VARCHAR contains the following information: Publisher name in French
The column name date_published with the data type VARCHAR contains the following information: The date of this open data published
The column name language with the data type VARCHAR contains the following information: What language this open data
The column name size with the data type BIGINT contains the following information: The open data size
The column name program_alignment_architecture_en with the data type VARCHAR contains the following information: English name of the program alignment architecture
The column name program_alignment_architecture_fr with the data type VARCHAR contains the following information: French name of the program alignment architecture
The column name date_released with the data type VARCHAR contains the following information: The date this open data released
The column name portal_url_en with the data type VARCHAR contains the following information: English portal url 
The column name portal_url_fr with the data type VARCHAR contains the following information: French portal url 
The column name user_votes with the data type BIGINT contains the following information: The users votes count for this open data, which showing which open data are most request by the user
The column name owner_org with the data type VARCHAR contains the following information: Which org divison are the owner of this open data
The column name owner_org_title with the data type VARCHAR contains the following information: The org division title of the owner of this open data

Your job is to write an execute a query that answers the following question:
{query}
"""

prompt = PromptTemplate.from_template(template)

agent_executor.run(
    prompt.format(query = "Total records of inventory table?")
)


llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /home/wei/Developments/llms/llama.cpp/models/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.f



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 'cdn_open_data'


llama_print_timings:        load time = 154034.90 ms
llama_print_timings:      sample time =    24.21 ms /    25 runs   (    0.97 ms per token,  1032.46 tokens per second)
llama_print_timings: prompt eval time = 186792.07 ms /  1226 tokens (  152.36 ms per token,     6.56 tokens per second)
llama_print_timings:        eval time =  5713.36 ms /    24 runs   (  238.06 ms per token,     4.20 tokens per second)
llama_print_timings:       total time = 192631.98 ms
Llama.generate: prefix-match hit


Action: sql_db_list_tables
Action Input: 'cdn_open_data'
Observation: inventory
Thought: Now I have the name of the table I will query, but I still need to check if my query is correct.
Action: sql_db_query_checker
Action Input: "SELECT COUNT(*) FROM inventory"


llama_print_timings:        load time = 154034.90 ms
llama_print_timings:      sample time =    49.18 ms /    52 runs   (    0.95 ms per token,  1057.38 tokens per second)
llama_print_timings: prompt eval time =  1072.36 ms /     6 tokens (  178.73 ms per token,     5.60 tokens per second)
llama_print_timings:        eval time = 12785.54 ms /    51 runs   (  250.70 ms per token,     3.99 tokens per second)
llama_print_timings:       total time = 14112.77 ms
Llama.generate: prefix-match hit


...


Observation: 

SELECT COUNT(*) FROM inventory

Note: The above-mentioned common mistakes were not found in the given query, so the original query is reproduced as is.
Thought: Finally, I will execute my query to get the result.
Action: sql_db_query
Action Input: "SELECT COUNT(*) FROM inventory" Finally, I will execute my query to get the result.
Action: sql_db_query
Action Input: "SELECT COUNT(*) FROM inventory"
Observation: [(11161,)]
Thought:


llama_print_timings:        load time = 154034.90 ms
llama_print_timings:      sample time =    33.59 ms /    37 runs   (    0.91 ms per token,  1101.49 tokens per second)
llama_print_timings: prompt eval time = 218471.57 ms /  1350 tokens (  161.83 ms per token,     6.18 tokens per second)
llama_print_timings:        eval time =  9078.99 ms /    36 runs   (  252.19 ms per token,     3.97 tokens per second)
llama_print_timings:       total time = 227728.92 ms
Llama.generate: prefix-match hit


 The final answer is 11161 records of inventory table.
Final Answer: 11161 records of inventory table. The final answer is 11161 records of inventory table.
Final Answer: 11161 records of inventory table.

> Finished chain.



llama_print_timings:        load time = 154034.90 ms
llama_print_timings:      sample time =    27.80 ms /    31 runs   (    0.90 ms per token,  1114.91 tokens per second)
llama_print_timings: prompt eval time =  2371.85 ms /    14 tokens (  169.42 ms per token,     5.90 tokens per second)
llama_print_timings:        eval time =  7808.43 ms /    30 runs   (  260.28 ms per token,     3.84 tokens per second)
llama_print_timings:       total time = 10358.93 ms


'11161 records of inventory table.'

In [7]:
agent_executor.run(
    prompt.format(query = "What's most released org division title?")
)



> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit


Action: sql_db_schema
Action Input: 'inventory'Action: sql_db_schema
Action Input: 'inventory'
Observation: Error: table_names {"'inventory'"} not found in database
Thought:


llama_print_timings:        load time = 154034.90 ms
llama_print_timings:      sample time =     8.71 ms /    19 runs   (    0.46 ms per token,  2182.40 tokens per second)
llama_print_timings: prompt eval time =  6534.77 ms /    49 tokens (  133.36 ms per token,     7.50 tokens per second)
llama_print_timings:        eval time =  3750.30 ms /    18 runs   (  208.35 ms per token,     4.80 tokens per second)
llama_print_timings:       total time = 10340.71 ms
Llama.generate: prefix-match hit


 The table is not valid.  I need to make sure I am looking at the correct table.  I will check the list of tables and see if there are any other tables that have an inventory column that could be relevant.
Action: sql_db_list_tables


llama_print_timings:        load time = 154034.90 ms
llama_print_timings:      sample time =    29.48 ms /    60 runs   (    0.49 ms per token,  2035.00 tokens per second)
llama_print_timings: prompt eval time =  2742.72 ms /    20 tokens (  137.14 ms per token,     7.29 tokens per second)
llama_print_timings:        eval time = 12780.71 ms /    59 runs   (  216.62 ms per token,     4.62 tokens per second)
llama_print_timings:       total time = 15715.82 ms


OutputParserException: Could not parse LLM output: ` The table is not valid.  I need to make sure I am looking at the correct table.  I will check the list of tables and see if there are any other tables that have an inventory column that could be relevant.
Action: sql_db_list_tables`

In [6]:
agent_executor.run(
    prompt.format(query = "what is the most voted open date between 2000 and 2023?")
)



> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit


Action: sql_db_schema
... (this Thought/Action/Action Input/Observation can repeat N times)



llama_print_timings:        load time = 115236.72 ms
llama_print_timings:      sample time =    44.55 ms /    33 runs   (    1.35 ms per token,   740.79 tokens per second)
llama_print_timings: prompt eval time = 12838.93 ms /    55 tokens (  233.44 ms per token,     4.28 tokens per second)
llama_print_timings:        eval time = 10637.32 ms /    32 runs   (  332.42 ms per token,     3.01 tokens per second)
llama_print_timings:       total time = 23701.60 ms


OutputParserException: Could not parse LLM output: `Action: sql_db_schema
... (this Thought/Action/Action Input/Observation can repeat N times)
`

In [8]:
agent_executor.run(
    prompt.format(query = "can you show me the top 10 open date in french title which have been published but never been released?")
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: inventory
Thought: I should query the schema of the inventory table to see what columns I can use.
Action: sql_db_schema
Action Input: inventory
Observation: 
CREATE TABLE inventory (
	ref_number VARCHAR, 
	title_en VARCHAR, 
	title_fr VARCHAR, 
	description_en VARCHAR, 
	description_fr VARCHAR, 
	publisher_en VARCHAR, 
	publisher_fr VARCHAR, 
	date_published VARCHAR, 
	language VARCHAR, 
	size BIGINT, 
	eligible_for_release VARCHAR, 
	program_alignment_architecture_en VARCHAR, 
	program_alignment_architecture_fr VARCHAR, 
	date_released VARCHAR, 
	portal_url_en VARCHAR, 
	portal_url_fr VARCHAR, 
	user_votes BIGINT, 
	owner_org VARCHAR, 
	owner_org_title VARCHAR
)

/*
3 rows from inventory table:
ref_number	title_en	title_fr	description_en	description_fr	publisher_en	publisher_fr	date_published	language	size	eligible_for_release	program_alignment_architecture_en	program_alignment_architecture

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-8QRUoez80xE1pV7j8qlCobzS on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-8QRUoez80xE1pV7j8qlCobzS on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

 I now know the final answer.
Final Answer: The top 10 open data in French title that have been published but never been released are: Rendu de décisions pour le comité sur la représentation à l'étranger (CORA), Requêtes de transport aux missions, Biens des missions et demandes de matériel, Requêtes d'informations financières et budgétaires des missions, Adhérence au standard de service de livraison pour les requêtes de services en ligne pour les missions (SLM), Niveau de satisfaction au service en ligne pour les missions, Liste des véhicules des services communs, SCDATA PSD 2023-2025 FINAL, Compte d’unités de logement résidentiel du ministère de la Défense nationale au Canada., and États financiers 21-22.

> Finished chain.


"The top 10 open data in French title that have been published but never been released are: Rendu de décisions pour le comité sur la représentation à l'étranger (CORA), Requêtes de transport aux missions, Biens des missions et demandes de matériel, Requêtes d'informations financières et budgétaires des missions, Adhérence au standard de service de livraison pour les requêtes de services en ligne pour les missions (SLM), Niveau de satisfaction au service en ligne pour les missions, Liste des véhicules des services communs, SCDATA PSD 2023-2025 FINAL, Compte d’unités de logement résidentiel du ministère de la Défense nationale au Canada., and États financiers 21-22."